In [49]:
import pandas as pd
import numpy as np
import regex as re
import matplotlib.pyplot as plt
import glob, os
import time
from datetime import datetime
import spacy

In [50]:
df = pd.read_excel(r'./Appendix_A_Capstone_DataSharingProposal.xlsx', sheet_name='A.25_ServiceNow_Incidents') # provide path for single .csv file
# directory = r'C:\Users\RJ\Downloads\2021_1021_re_esg\\' # or provide a directory that contains a collection of .csv/.xlsx files:

start_time = datetime.now()

In [51]:
# source_type = r'csv' # can be: directory, csv, or xlsx
# if source_type == 'directory':
#     df = pd.concat(map(lambda file: pd.read_csv(file, sep= ";", encoding = "utf-8-sig", low_memory=False), \
#                        glob.glob(os.path.join('', directory + '*.csv'))))
# elif source_type == r'csv':
#     df = pd.read_csv(csv, sep=';', low_memory=False)
# elif source_type == r'xlsx':
#     sheet_name = 1
#     df = pd.read_excel(path, sheet_name=sheet_name)

In [52]:
output_dir = r'./output'

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Columns: 118 entries, parent to category
dtypes: bool(9), float64(58), int64(14), object(37)
memory usage: 10.4+ KB


In [54]:
for col in df.columns:
    print(col)

parent
caused_by
watch_list
u_browser_type
u_swap
upon_reject
sys_updated_on
origin_table
skills
u_gl_code
number
u_moniker
state
knowledge
order
contract
impact
active
priority
sys_domain_path
u_asset_substatus
business_duration
group_list
approval_set
u_public_pc
needs_attention
universal_request
short_description
correlation_display
u_error_code
work_start
additional_assignee_list
u_requestor
notify
service_offering
sys_class_name
closed_by
follow_up
parent_incident
reopened_by
u_hardware_swap
reassignment_count
u_end_of_lifecycle
assigned_to
sla_due
comments_and_work_notes
u_actual_resolution_time
u_affect_asset
u_replacement_asset
u_cause_code
escalation
upon_approval
correlation_id
u_location
made_sla
u_browsing_data_cleared
u_paging_response
child_incidents
task_effective_number
resolved_by
sys_updated_by
opened_by
user_input
sys_created_on
sys_domain
route_reason
calendar_stc
closed_at
u_error_code_account_alerts
u_windows_version
business_service
business_impact
rfc
time_worke


### Named Entity Recognition (NER)


Sizes: ['sm', 'md', 'lg']<br>
Your pipeline must be compatable with your current version of SpaCy.

Can download the following on Conda (base) environment:
`pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_[SIZE]-[VERSION]/en_core_web_md-[VERSION].tar.gz`

Alternatively, in Python:
`python -m spacy download en_core_web_sm`

In [55]:
from spacy.lang.en import English
nlp = English()

# Instantiate a Tokenizer with the default settings for English, including punctuation rules and exceptions.
tokenizer = nlp.tokenizer

In [56]:
# Assign SpaCy `en_core_web_[SIZE]` as `nlp`.
nlp = spacy.load('en_core_web_sm')

In [57]:
# Example of token generation for the first body of text.
doc = nlp(df['description'][0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_) # Print: token, POS, syntactic dependency.

not getting a DHCP address from server
Send Pro P2000
MAC: C400AD3CDBE9

PB Tech Cell #: <PHONE NUMBER> - <NAME>
x4049
not PART neg
getting VERB ROOT
a DET det
DHCP PROPN compound
address NOUN dobj
from ADP prep
server NOUN pobj

 SPACE dep
Send VERB acl
Pro PROPN nmod
P2000 PROPN nmod

 SPACE dep
MAC PROPN dobj
: PUNCT punct
C400AD3CDBE9 ADJ amod


 SPACE dep
PB PROPN compound
Tech PROPN compound
Cell PROPN dep
# NOUN advmod
: PUNCT punct
< X punct
PHONE NOUN compound
NUMBER PROPN dep
> X punct
- PUNCT punct
< X nmod
NAME PROPN appos
> PUNCT punct

 SPACE dep
x4049 NOUN punct


In [58]:
# Create function to add an article's tokens to `doc_list`.
# Tokenize one time, then use that object for the subsequent accumulators.
# Returns None many times.
doc_list = []
def to_doc_list(text):
    doc_list.append(nlp(text))

In [59]:
# Takes time to generate tokens from each cell's fulltext.
df['description'].apply(to_doc_list)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
Name: description, dtype: object

In [60]:
# Assign `doc_list` to `doc_series` as a Series object.
doc_series = pd.Series(doc_list)
doc_series

0     (not, getting, a, DHCP, address, from, server,...
1       (Can, not, connect, to, in, -, office, Desktop)
2     (received, from, :, <, EMAIL, >, \n\n, Good, m...
3     (Docking, station, not, providing, monitor, di...
4     (Baird, TrustDesk, Migration, to, OneDrive, -,...
5                      (Deposit, Edge, Account, Unlock)
6     (User, is, receiving, a, password, prompt, for...
7     (Beta, report, ran, out, of, paper, ., Needs, ...
8         (Laptop, not, charging, on, Docking, Station)
9               (Mac, not, connecting, to, Guest, wifi)
10    (<, NAME, >, needs, his, Outlook, inbox, resto...
11    (Hello, ., My, name, is, spelled, incorrectly,...
dtype: object

In [61]:
for doc in doc_series:
    filtered_string = ""
    for token in doc:
        if token.pos_ in ['PROPN', 'NUM']:
            new_token = " <{}>".format(token.ent_type_)
        elif token.pos_ == "PUNCT":
            new_token = token.text
        else:
            new_token = " {}".format(token.text)
        filtered_string += new_token
    filtered_string = filtered_string[1:]
    print(filtered_string)

not getting a <> address from server 
 Send <> <> 
 <ORG>: C400AD3CDBE9 

 <> <> <> #: < PHONE <> >- < <>> 
 x4049
Can not connect to in- office Desktop
received from: < <> > 

 Good morning, 

 If possible, please could someone come and help < NAME > with his displays- he is using <CARDINAL> of the <> <> desks upstairs. 

 Thanks 

 < <> > 

 <> <> 
 <> <> 
 <> <> <> 
( o)   +44 <MONEY> <CARDINAL> <PERSON> 
 <>> |   bairdeurope.com 

[ Graphical user interface     <> automatically generated]<http://www.rwbaird.com/ >
Docking station not providing monitor display or network 

 < <> > is borrowing another docking station. < <> > <> will not light up whatsoever when plugged in. 

<ORG> <ORG> <ORG> to <ORG>- Error migrating 
 There was an error verifying your <ORG> migration, please do not try to Use <GPE> or <> and contact the <> <>.
<> <> <> <>
<> is receiving a password prompt for their <DATE> day re - approval for email access. <ORG> authenticator is looping and will not authenticate.


Dictionary accumulator for entities based on entity type


In [62]:
ent_dict = {}
def count_ent(doc):
    for ent in doc.ents:
        if ent.label_ not in ent_dict:
            ent_dict[ent.label_] = 1
        else:
            ent_dict[ent.label_]+=1
doc_series.apply(count_ent)
ent_dict

{'ORG': 7,
 'CARDINAL': 3,
 'TIME': 1,
 'MONEY': 1,
 'PERSON': 2,
 'GPE': 1,
 'DATE': 1,
 'PRODUCT': 1}


# Check NER for misses/gaps.



#### PERSON.


In [63]:
person_dict = {}
def count_person(doc):
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            if ent.text not in person_dict:
                person_dict[ent.text]=1
            else:
                person_dict[ent.text]+=1
doc_series.apply(count_person)
sorted(person_dict.items(), key=lambda x: x[1], reverse=True)[0:10]

[('8239', 1), ('Guest', 1)]


#### GPE (geopolitical entity).


In [64]:
gpe_dict = {}
def count_gpe(doc):
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in gpe_dict:
                gpe_dict[ent.text]=1
            else:
                gpe_dict[ent.text]+=1
doc_series.apply(count_gpe)
sorted(gpe_dict.items(), key=lambda x: x[1], reverse=True)[0:30]

[('Use Trustdesk', 1)]


#### ORG (organization).


In [65]:
org_dict = {}
def count_org(doc):
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            if ent.text not in org_dict:
                org_dict[ent.text]=1
            else:
                org_dict[ent.text]+=1
doc_series.apply(count_org)
sorted(org_dict.items(), key=lambda x: x[1], reverse=True)[0:10]

[('MAC', 1),
 ('+44', 1),
 ('Baird TrustDesk Migration', 1),
 ('OneDrive - Error', 1),
 ('TrustDesk', 1),
 ('MFA', 1),
 ('DocuSign', 1)]


#### FAC (factory).


In [66]:
fac_dict = {}
def count_fac(doc):
    for ent in doc.ents:
        if ent.label_ == 'FAC':
            if ent.text not in fac_dict:
                fac_dict[ent.text]=1
            else:
                fac_dict[ent.text]+=1
doc_series.apply(count_fac)
sorted(fac_dict.items(), key=lambda x: x[1], reverse=True)[0:10]

[]


#### TIME.


In [67]:
time_dict = {}
def count_time(doc):
    for ent in doc.ents:
        if ent.label_ == 'TIME':
            if ent.text not in time_dict:
                time_dict[ent.text]=1
            else:
                time_dict[ent.text]+=1
doc_series.apply(count_time)
sorted(time_dict.items(), key=lambda x: x[1], reverse=True)[0:10]

[('morning', 1)]


#### EVE (event).


In [68]:
event_dict = {}
def count_event(doc):
    for ent in doc.ents:
        if ent.label_ == 'EVENT':
            if ent.text not in event_dict:
                event_dict[ent.text]=1
            else:
                event_dict[ent.text]+=1
doc_series.apply(count_event)
sorted(event_dict.items(), key=lambda x: x[1], reverse=True)[0:10]

[]


___

&nbsp;
#### Save out any of the above as a Series object


In [69]:
ner_obj = sorted(org_dict.items(), key=lambda x: x[1], reverse=True) # specify which dict to save
ner_obj = pd.Series(ner_obj)
ner_obj
# ner_obj.to_csv(output_dir+'ner_obj.csv', sep=';', encoding='utf-8-sig')

0                          (MAC, 1)
1                          (+44, 1)
2    (Baird TrustDesk Migration, 1)
3             (OneDrive - Error, 1)
4                    (TrustDesk, 1)
5                          (MFA, 1)
6                     (DocuSign, 1)
dtype: object


***
